## ⭐ Star Type Prediction

Given *data about stars*, let's try to predict the **type** of a given star.

We will use a logistic regression model to make our predictions and evaluate the model performance using both test set and k-fold evaluation.

Data source: https://www.kaggle.com/datasets/deepu1109/star-dataset

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression

import warnings
warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv('6 class csv.csv')
data

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3068,0.002400,0.1700,16.12,0,Red,M
1,3042,0.000500,0.1542,16.60,0,Red,M
2,2600,0.000300,0.1020,18.70,0,Red,M
3,2800,0.000200,0.1600,16.65,0,Red,M
4,1939,0.000138,0.1030,20.06,0,Red,M
...,...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,5,Blue,O
236,30839,834042.000000,1194.0000,-10.63,5,Blue,O
237,8829,537493.000000,1423.0000,-10.73,5,White,A
238,9235,404940.000000,1112.0000,-11.23,5,White,A


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Temperature (K)         240 non-null    int64  
 1   Luminosity(L/Lo)        240 non-null    float64
 2   Radius(R/Ro)            240 non-null    float64
 3   Absolute magnitude(Mv)  240 non-null    float64
 4   Star type               240 non-null    int64  
 5   Star color              240 non-null    object 
 6   Spectral Class          240 non-null    object 
dtypes: float64(3), int64(2), object(2)
memory usage: 13.2+ KB


### Preprocessing

In [4]:
df = data.copy()

In [5]:
pd.get_dummies(df['Star color'], dtype=int)

,Blue,Blue,Blue White,Blue white,Blue white,Blue-White,Blue-white,Orange,Orange-Red,Pale yellow orange,Red,White,White-Yellow,Whitish,Yellowish,Yellowish White,white,yellow-white,yellowish
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
236,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
237,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
238,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [6]:
df['Star color'].unique()

array(['Red', 'Blue White', 'White', 'Yellowish White', 'Blue white',
       'Pale yellow orange', 'Blue', 'Blue-white', 'Whitish',
       'yellow-white', 'Orange', 'White-Yellow', 'white', 'Blue ',
       'yellowish', 'Yellowish', 'Orange-Red', 'Blue white ',
       'Blue-White'], dtype=object)

In [7]:
# Fix color values
color_mapping = {
    "Blue ": "Blue",
    "Blue white": "Blue White",
    "Blue-white": "Blue White",
    "Blue white ": "Blue White",
    "Blue-White": "Blue White",
    "white": "White",
    "yellow-white": "Yellowish White",
    "White-Yellow": "Yellowish White",
    "yellowish": "Yellowish"
}

df["Star color"] = df['Star color'].replace(color_mapping)

In [8]:
df['Star color'].unique()

array(['Red', 'Blue White', 'White', 'Yellowish White',
       'Pale yellow orange', 'Blue', 'Whitish', 'Orange', 'Yellowish',
       'Orange-Red'], dtype=object)

In [9]:
pd.get_dummies(df['Star color'], dtype=int)

,Blue,Blue White,Orange,Orange-Red,Pale yellow orange,Red,White,Whitish,Yellowish,Yellowish White
0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
235,1,0,0,0,0,0,0,0,0,0
236,1,0,0,0,0,0,0,0,0,0
237,0,0,0,0,0,0,1,0,0,0
238,0,0,0,0,0,0,1,0,0,0


In [10]:
def onehot_encode(df, column, prefix):
    dummies = pd.get_dummies(df[column], prefix=prefix, dtype=int)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [11]:
# One-hot encode
df = onehot_encode(df, 'Star color', 'Color')
df = onehot_encode(df, 'Spectral Class', 'Class')
df

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Color_Blue,Color_Blue White,Color_Orange,Color_Orange-Red,Color_Pale yellow orange,...,Color_Whitish,Color_Yellowish,Color_Yellowish White,Class_A,Class_B,Class_F,Class_G,Class_K,Class_M,Class_O
0,3068,0.002400,0.1700,16.12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,3042,0.000500,0.1542,16.60,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2600,0.000300,0.1020,18.70,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2800,0.000200,0.1600,16.65,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1939,0.000138,0.1030,20.06,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
236,30839,834042.000000,1194.0000,-10.63,5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
237,8829,537493.000000,1423.0000,-10.73,5,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
238,9235,404940.000000,1112.0000,-11.23,5,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [12]:
# Split df into X and y
y = df['Star type']
X = df.drop('Star type', axis=1)

In [13]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

In [14]:
X_train.shape, X_test.shape

((168, 21), (72, 21))

In [15]:
X_train

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Color_Blue,Color_Blue White,Color_Orange,Color_Orange-Red,Color_Pale yellow orange,Color_Red,...,Color_Whitish,Color_Yellowish,Color_Yellowish White,Class_A,Class_B,Class_F,Class_G,Class_K,Class_M,Class_O
182,3419,0.000245,0.12600,17.56,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
59,3535,195000.000000,1546.00000,-11.36,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
168,17383,342900.000000,30.00000,-6.09,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
221,12749,332520.000000,76.00000,-7.02,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
138,3324,0.003400,0.34000,12.23,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,3598,0.001100,0.56000,14.26,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
72,3304,0.008500,0.18000,13.20,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
140,13420,0.000590,0.00981,13.67,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
235,38940,374830.000000,1356.00000,-9.93,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
X_train.describe()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Color_Blue,Color_Blue White,Color_Orange,Color_Orange-Red,Color_Pale yellow orange,Color_Red,...,Color_Whitish,Color_Yellowish,Color_Yellowish White,Class_A,Class_B,Class_F,Class_G,Class_K,Class_M,Class_O
count,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.0,168.000000,...,168.0,168.000000,168.000000,168.000000,168.000000,168.000000,168.0,168.000000,168.000000,168.000000
mean,10466.922619,102646.730105,189.139364,4.991393,0.226190,0.208333,0.011905,0.005952,0.0,0.470238,...,0.0,0.005952,0.029762,0.077381,0.226190,0.047619,0.0,0.017857,0.476190,0.154762
std,9199.213104,183562.568545,468.615093,10.439533,0.419615,0.407331,0.108782,0.077152,0.0,0.500606,...,0.0,0.077152,0.170438,0.267994,0.419615,0.213596,0.0,0.132828,0.500926,0.362759
min,2600.000000,0.000085,0.008400,-11.920000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,3341.750000,0.000817,0.107250,-6.105000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
50%,6568.500000,0.008300,0.501000,10.635000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
75%,15554.500000,171000.000000,31.250000,13.990000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000
max,40000.000000,849420.000000,1779.000000,20.020000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,...,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000


In [17]:
# Scale X
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [18]:
X_train

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Color_Blue,Color_Blue White,Color_Orange,Color_Orange-Red,Color_Pale yellow orange,Color_Red,...,Color_Whitish,Color_Yellowish,Color_Yellowish White,Class_A,Class_B,Class_F,Class_G,Class_K,Class_M,Class_O
0,-0.768434,-0.560864,-0.404550,1.207543,-0.540655,-0.512989,-0.109764,-0.077382,0.0,1.061406,...,0.0,-0.077382,-0.175142,-0.289605,-0.540655,-0.223607,0.0,-0.13484,1.048809,-0.427900
1,-0.755787,0.504620,2.904125,-1.570978,-0.540655,-0.512989,-0.109764,-0.077382,0.0,1.061406,...,0.0,-0.077382,-0.175142,-0.289605,-0.540655,-0.223607,0.0,-0.13484,1.048809,-0.427900
2,0.754059,1.312749,-0.340610,-1.064657,1.849609,-0.512989,-0.109764,-0.077382,0.0,-0.942147,...,0.0,-0.077382,-0.175142,-0.289605,-0.540655,-0.223607,0.0,-0.13484,-0.953463,2.336993
3,0.248815,1.256032,-0.242155,-1.154008,1.849609,-0.512989,-0.109764,-0.077382,0.0,-0.942147,...,0.0,-0.077382,-0.175142,-0.289605,-0.540655,-0.223607,0.0,-0.13484,-0.953463,2.336993
4,-0.778792,-0.560864,-0.404092,0.695457,-0.540655,-0.512989,-0.109764,-0.077382,0.0,1.061406,...,0.0,-0.077382,-0.175142,-0.289605,-0.540655,-0.223607,0.0,-0.13484,1.048809,-0.427900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,-0.748918,-0.560864,-0.403622,0.890492,-0.540655,-0.512989,-0.109764,-0.077382,0.0,1.061406,...,0.0,-0.077382,-0.175142,-0.289605,-0.540655,-0.223607,0.0,-0.13484,1.048809,-0.427900
164,-0.780973,-0.560864,-0.404435,0.788651,-0.540655,-0.512989,-0.109764,-0.077382,0.0,1.061406,...,0.0,-0.077382,-0.175142,-0.289605,-0.540655,-0.223607,0.0,-0.13484,1.048809,-0.427900
165,0.321974,-0.560864,-0.404799,0.833807,-0.540655,1.949359,-0.109764,-0.077382,0.0,-0.942147,...,0.0,-0.077382,-0.175142,-0.289605,1.849609,-0.223607,0.0,-0.13484,-0.953463,-0.427900
166,3.104418,1.487215,2.497463,-1.433589,1.849609,-0.512989,-0.109764,-0.077382,0.0,-0.942147,...,0.0,-0.077382,-0.175142,-0.289605,-0.540655,-0.223607,0.0,-0.13484,-0.953463,2.336993


In [19]:
X_train.describe()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Color_Blue,Color_Blue White,Color_Orange,Color_Orange-Red,Color_Pale yellow orange,Color_Red,...,Color_Whitish,Color_Yellowish,Color_Yellowish White,Class_A,Class_B,Class_F,Class_G,Class_K,Class_M,Class_O
count,1.680000e+02,1.680000e+02,168.000000,1.680000e+02,1.680000e+02,1.680000e+02,1.680000e+02,1.680000e+02,168.0,1.680000e+02,...,168.0,1.680000e+02,1.680000e+02,1.680000e+02,168.000000,1.680000e+02,168.0,1.680000e+02,1.680000e+02,1.680000e+02
mean,6.344132e-17,-1.163091e-16,0.000000,-1.057355e-17,2.643388e-17,-6.872809e-17,-3.172066e-17,2.643388e-17,0.0,5.286776e-18,...,0.0,3.700743e-17,4.229421e-17,3.172066e-17,0.000000,-3.172066e-17,0.0,7.930164e-18,1.850372e-17,-6.344132e-17
std,1.002990e+00,1.002990e+00,1.002990,1.002990e+00,1.002990e+00,1.002990e+00,1.002990e+00,1.002990e+00,0.0,1.002990e+00,...,0.0,1.002990e+00,1.002990e+00,1.002990e+00,1.002990,1.002990e+00,0.0,1.002990e+00,1.002990e+00,1.002990e+00
min,-8.577300e-01,-5.608638e-01,-0.404802,-1.624781e+00,-5.406549e-01,-5.129892e-01,-1.097643e-01,-7.738232e-02,0.0,-9.421467e-01,...,0.0,-7.738232e-02,-1.751424e-01,-2.896048e-01,-0.540655,-2.236068e-01,0.0,-1.348400e-01,-9.534626e-01,-4.279002e-01
25%,-7.768571e-01,-5.608638e-01,-0.404591,-1.066098e+00,-5.406549e-01,-5.129892e-01,-1.097643e-01,-7.738232e-02,0.0,-9.421467e-01,...,0.0,-7.738232e-02,-1.751424e-01,-2.896048e-01,-0.540655,-2.236068e-01,0.0,-1.348400e-01,-9.534626e-01,-4.279002e-01
50%,-4.250447e-01,-5.608637e-01,-0.403748,5.422157e-01,-5.406549e-01,-5.129892e-01,-1.097643e-01,-7.738232e-02,0.0,-9.421467e-01,...,0.0,-7.738232e-02,-1.751424e-01,-2.896048e-01,-0.540655,-2.236068e-01,0.0,-1.348400e-01,-9.534626e-01,-4.279002e-01
75%,5.546982e-01,3.734836e-01,-0.337935,8.645510e-01,-5.406549e-01,-5.129892e-01,-1.097643e-01,-7.738232e-02,0.0,1.061406e+00,...,0.0,-7.738232e-02,-1.751424e-01,-2.896048e-01,-0.540655,-2.236068e-01,0.0,-1.348400e-01,1.048809e+00,-4.279002e-01
max,3.219989e+00,4.080384e+00,3.402822,1.443890e+00,1.849609e+00,1.949359e+00,9.110434e+00,1.292285e+01,0.0,1.061406e+00,...,0.0,1.292285e+01,5.709641e+00,3.452981e+00,1.849609,4.472136e+00,0.0,7.416198e+00,1.048809e+00,2.336993e+00


In [20]:
y_train

182    0
59     5
168    4
221    4
138    1
      ..
137    1
72     1
140    2
235    5
37     3
Name: Star type, Length: 168, dtype: int64

### Training

#### Test Set Evaluation

In [21]:
model = LogisticRegression()
model.fit(X_train, y_train)

print("Test Set Accuracy: {:.2f}%".format(model.score(X_test, y_test) * 100))

Test Set Accuracy: 95.83%


#### K-Fold Evaluation

In [35]:
# Scale X
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), columns=X.columns)

In [36]:
kf = KFold(n_splits=5)

print("Split Indices")

for i, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"\nSplit {i+1}: \n-----------")
    print("\nTrain:\n" + str(train_idx))
    print("\nTest:\n" + str(test_idx) + "\n")

Split Indices

Split 1: 
-----------

Train:
[ 48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155
 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173
 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191
 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209
 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227
 228 229 230 231 232 233 234 235 236 237 238 239]

Test:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]


Split 2: 
---------

In [37]:
X.iloc[train_idx, :]

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Color_Blue,Color_Blue White,Color_Orange,Color_Orange-Red,Color_Pale yellow orange,Color_Red,...,Color_Whitish,Color_Yellowish,Color_Yellowish White,Class_A,Class_B,Class_F,Class_G,Class_K,Class_M,Class_O
0,-0.779382,-0.598624,-0.459210,1.116745,-0.551677,-0.453905,-0.09167,-0.064685,-0.064685,1.069045,...,-0.09167,-0.112509,-0.229416,-0.293211,-0.486943,-0.276104,-0.064685,-0.160128,1.078036,-0.447214
1,-0.782110,-0.598624,-0.459241,1.162414,-0.551677,-0.453905,-0.09167,-0.064685,-0.064685,1.069045,...,-0.09167,-0.112509,-0.229416,-0.293211,-0.486943,-0.276104,-0.064685,-0.160128,1.078036,-0.447214
2,-0.828477,-0.598624,-0.459342,1.362213,-0.551677,-0.453905,-0.09167,-0.064685,-0.064685,1.069045,...,-0.09167,-0.112509,-0.229416,-0.293211,-0.486943,-0.276104,-0.064685,-0.160128,1.078036,-0.447214
3,-0.807496,-0.598624,-0.459229,1.167171,-0.551677,-0.453905,-0.09167,-0.064685,-0.064685,1.069045,...,-0.09167,-0.112509,-0.229416,-0.293211,-0.486943,-0.276104,-0.064685,-0.160128,1.078036,-0.447214
4,-0.897819,-0.598624,-0.459340,1.491607,-0.551677,-0.453905,-0.09167,-0.064685,-0.064685,1.069045,...,-0.09167,-0.112509,-0.229416,-0.293211,-0.486943,-0.276104,-0.064685,-0.160128,1.078036,-0.447214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.735742,-0.598624,-0.459346,1.169074,-0.551677,-0.453905,-0.09167,-0.064685,-0.064685,1.069045,...,-0.09167,-0.112509,-0.229416,-0.293211,-0.486943,-0.276104,-0.064685,-0.160128,1.078036,-0.447214
188,-0.809804,-0.598624,-0.459322,1.433570,-0.551677,-0.453905,-0.09167,-0.064685,-0.064685,1.069045,...,-0.09167,-0.112509,-0.229416,-0.293211,-0.486943,-0.276104,-0.064685,-0.160128,1.078036,-0.447214
189,-0.731651,-0.598624,-0.459289,1.138628,-0.551677,-0.453905,-0.09167,-0.064685,-0.064685,1.069045,...,-0.09167,-0.112509,-0.229416,-0.293211,-0.486943,-0.276104,-0.064685,-0.160128,1.078036,-0.447214
190,-0.738050,-0.598624,-0.459253,0.674333,-0.551677,-0.453905,-0.09167,-0.064685,-0.064685,1.069045,...,-0.09167,-0.112509,-0.229416,-0.293211,-0.486943,-0.276104,-0.064685,-0.160128,1.078036,-0.447214


In [38]:
results = []

for train_idx, test_idx in kf.split(X):
    train_set = (X.iloc[train_idx, :], y.iloc[train_idx])
    test_set = (X.iloc[test_idx, :], y.iloc[test_idx])

    model = LogisticRegression()
    model.fit(train_set[0], train_set[1])
    results.append(model.score(test_set[0], test_set[1]))

In [40]:
print("K-Fold Accuracies: ")
for i, result in enumerate(results):
    print("Model {}: {:.2f}%".format(i+1, result*100))

K-Fold Accuracies: 
Model 1: 83.33%
Model 2: 95.83%
Model 3: 100.00%
Model 4: 97.92%
Model 5: 89.58%


In [44]:
print("Average K-Fold Accuracy: {:.2f}%".format(np.array(results).mean()*100))

Average K-Fold Accuracy: 93.33%
